# HW 3
- Author: Marc Brooks
- NetID: mgb45

## 1.) (Hard-thresholding)

## 2. (Great British Bake-off)

In [94]:
# importing required packages
import numpy as np
import pandas as pd
import itertools

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LassoLarsIC, LassoLarsCV, RidgeCV, LinearRegression, Ridge
from sklearn.model_selection import cross_val_score, LeaveOneOut
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")

## (a)

First, since $\epsilon$ and $\textbf{X}$ are independent we can see that: 
$$
\begin{align}
\text{Var}(Y) &= \text{Var}(\textbf{X}^{T}\beta^{*} + \epsilon) \\
&= \text{Var}(\textbf{X}^{T}\beta^{*}) + \text{Var}(\epsilon) \\
&= \beta^{*T}\Sigma(\rho)\beta^{*} + \sigma^2 
\end{align}
$$

Furthmore, we recognize $P(Y - \textbf{X}^{T}\beta^{*})^{2} = P\epsilon^{2}$ as the expected squared residuals.
Since $P\epsilon = 0$ then $P\epsilon^2 = \sigma^2$ and $P(Y - \textbf{X}^{T}\beta^{*})^{2} = \sigma^2$.


Thus,
$$
\begin{align}
R^{2} &= 1 - \frac{P(Y - \textbf{X}^{T}\beta^{*})^{2}}{\text{Var}(Y)} \\
&= 1 - \frac{\sigma^2}{\beta^{*T}\Sigma(\rho)\beta^{*} + \sigma^2}
\end{align}
$$

## (b)

Setting $R^2 = .8$ we can solve for $\sigma^2$ as follows.

$$
\begin{align}
.8 &= 1 - \frac{\sigma^2}{\beta^{*T}\Sigma(\rho)\beta^{*} + \sigma^2} \\
.2 &= \frac{\sigma^2}{\beta^{*T}\Sigma(\rho)\beta^{*} + \sigma^2} \\
\sigma^2 &= .2(\beta^{*T}\Sigma(\rho)\beta^{*} + \sigma^2)  \\
.8\sigma^2 &= .2(\beta^{*T}\Sigma(\rho)\beta^{*})  \\
\sigma^2 &= .25(\beta^{*T}\Sigma(\rho)\beta^{*}) 
\end{align}
$$

In [8]:
n = 100
p = [10, 25, 50]
rho = [0, .25, .5]

Generating data sets with sparse signal

In [9]:
rho = .25
p=10
n = 100

In [10]:
beta_sp = np.fromfunction(lambda j,_: (2/np.sqrt(n))*(j+1 <= np.sqrt(p)), (p,1))

In [11]:
beta_dns =  np.fromfunction(lambda j,_: (5/(j+1)*np.sqrt(n)), (p,1))

In [12]:
Sigma_p = np.fromfunction(lambda i,j: rho**(np.abs(i-j)), (p,p))

In [13]:
sigma2_spr = beta_sp.T @ Sigma_p @ beta_sp
sigma2_dns = beta_dns.T @ Sigma_p @ beta_dns

In [14]:
epsilon_spr = np.random.normal(loc=0, scale=sigma2_spr)
epsilon_dns = np.random.normal(loc=0, scale=sigma2_dns)

In [15]:
X50.

In [16]:
Y_spr =  X @ beta_sp + epsilon_spr
Y_dns =  X @ beta_dns + epsilon_dns

In [17]:
lasso_aic = LassoLarsIC(criterion='aic')
lasso_aic.fit(X,Y_spr.reshape(n,))

LassoLarsIC()

In [18]:
lasso_aic.score(X, Y_spr)

1.0

In [66]:
mean_squared_error(Y_spr, lasso_aic.predict(X))

1.2433803720963995e-32

In [67]:

loo = LeaveOneOut()
loo

LeaveOneOut()

In [87]:
%%time
lasso_loo = LassoLarsCV(cv=loo)
lasso_loo.fit(X,Y_spr.reshape(n,))

Wall time: 70 ms


LassoLarsCV(cv=LeaveOneOut())

In [86]:
%%time
lm_spr = LinearRegression().fit(X, Y_spr)

Wall time: 998 µs


In [70]:
lm_spr.coef_.round(5)

array([[ 0.2,  0.2,  0.2,  0. ,  0. , -0. , -0. , -0. , -0. ,  0. ]])

In [71]:
1/(lm_spr.coef_**2)**(1/2)

array([[5.00000000e+00, 5.00000000e+00, 5.00000000e+00, 5.33759956e+15,
        3.60287970e+16, 5.54289185e+15, 3.60287970e+16, 7.20575940e+16,
        1.10857837e+16, 1.44115188e+16]])

In [72]:
X.shape

(100, 10)

In [73]:
(X @ np.diagflat(np.abs(lm_spr.coef_))).shape

(100, 10)

In [4]:
labels = ['lasso']*3 + ['adpt lasso']*3 + ['ridge']*3 + ['adpt ridge']*3
tunings = ['AIC', 'BIC', 'LOO-CV']*4
labs = [labels[i] + ' ' + tunings[i] for i in range(len(labels))]

In [6]:
emse_table = pd.DataFrame(itertools.product(['sparse', 'dense'], 
              50.                 labs,
                               [10,25,50],
                               [0,.25,.5]), 
             columns=['signal','Method-Tuning', 'p', 'rho'])

emse_table['mse'] = None

In [7]:
emse_table

,signal,Method-Tuning,p,rho,mse
0,sparse,lasso AIC,10,0.00,None
1,sparse,lasso AIC,10,0.25,None
2,sparse,lasso AIC,10,0.50,None
3,sparse,lasso AIC,25,0.00,None
4,sparse,lasso AIC,25,0.25,None
...,...,...,...,...,...
211,dense,adpt ridge LOO-CV,25,0.25,None
212,dense,adpt ridge LOO-CV,25,0.50,None
213,dense,adpt ridge LOO-CV,50,0.00,None
214,dense,adpt ridge LOO-CV,50,0.25,None


In [77]:
U, D, V = np.linalg.svd(X, full_matrices=False)

In [78]:
(D**2/(D**2 + .02)).shape

(10,)

In [79]:
(U @ np.diag(D**2/(D**2 + .02)) @ U.T) @ Y_spr

array([[ 0.09131373],
       [-0.34360446],
       [-0.76722612],
       [-0.01278452],
       [-0.36367278],
       [-0.13878086],
       [-0.10490522],
       [-0.80090635],
       [ 0.27997823],
       [-1.1837209 ],
       [ 0.19109503],
       [-0.55550095],
       [ 0.18238372],
       [ 0.35403677],
       [ 0.37785675],
       [ 0.38293946],
       [ 0.06036646],
       [-0.0408259 ],
       [ 0.51215244],
       [ 0.16381114],
       [ 0.20305534],
       [-0.20862837],
       [-0.24778447],
       [ 0.10291171],
       [ 0.57958451],
       [ 0.04744084],
       [ 0.01521899],
       [ 0.30078279],
       [ 0.0792429 ],
       [ 0.1288274 ],
       [-0.03842772],
       [ 0.58528312],
       [-0.00970752],
       [-0.23951654],
       [-0.14492896],
       [-0.3822954 ],
       [ 0.50949841],
       [ 0.10597148],
       [ 0.34846273],
       [ 0.19697722],
       [ 0.54649506],
       [ 0.42254295],
       [ 0.52689629],
       [-0.2840093 ],
       [ 0.20744514],
       [ 0

In [95]:
def lambda_bic(lambs, design_mat, Y_mat):
    U, D, V = np.linalg.svd(design_mat, full_matrices=False)
    
    rss = lambda l: sum((Y_mat - (U @ np.diag(D**2/(D**2 + l)) @ U.T) @ Y_mat)**2)
    df_lambda = lambda l: sum(D**2/(D**2 + l)) 
    
    min_i = np.argmin([np.log(rss(l)) + df_lambda(l)*np.log(n)/n for l in lambs])
    
    mse = rss(lambs[min_i])/Y_mat.shape[0]
    
    return mse, lambs[min_i]

In [97]:
scaler = StandardScaler()

In [98]:
scaler.fit(X)

StandardScaler()

In [102]:
X_std = scaler.transform(X)

In [110]:
lambda_bic(np.linspace(0, 1, 100), X2, Y_dns)

(array([4.27135389e-24]), 0.0)

In [44]:
ridge = Ridge(alpha=.5)

In [45]:
ridge.fit(X, Y_dns)
mean_squared_error(Y_dns, ridge.predict(X))

0.08455723870325346

In [59]:
mean_squared_error(Y_dns, (U @ np.diag(D**2/(D**2 + .5)) @ U.T) @ Y_dns)

2104004.2531023617

In [113]:
mean_squared_error(Y_dns , X2 @ (np.linalg.inv(X2.T@X2 + .2*np.eye(X2.shape[1])) @ X2.T) @ Y_dns)

10.586349670190542

In [124]:
ridge = Ridge(alpha=.2, fit_intercept=False)
ridge.fit(X_std, Y_dns)
mean_squared_error(Y_dns, ridge.predict(X_std))

2342143.0593981217

In [122]:
U, D, V = np.linalg.svd(X_std, full_matrices=False)

mean_squared_error(Y_dns , (U @ np.diag(D**2/(D**2 + .2)) @ U.T) @ Y_dns)

2342143.0593981217

In [ ]:
from tqdm import tqdm

n = 100

for p,rho in itertools.product([10, 25, 50],[0, 0.25, 0.5]):
    print(p, rho)
    
    # Generate data
    beta_spr = np.fromfunction(lambda j, _: (2 / np.sqrt(n)) * (j + 1 <= np.sqrt(p)), (p, 1))
    beta_dns = np.fromfunction(lambda j, _: (5 / (j + 1) * np.sqrt(n)), (p, 1))

    Sigma_p = np.fromfunction(lambda i, j: rho**(np.abs(i - j)), (p, p))
    sigma2_spr = 0.25 * (beta_spr.T @ Sigma_p @ beta_spr)
    sigma2_dns = 0.25 * (beta_dns.T @ Sigma_p @ beta_dns)
    
    # Sparse Signal
    lasso_aic_mse_spr = 0
    lasso_bic_mse_spr = 0
    lasso_loocv_mse_spr = 0
    adp_lasso_aic_mse_spr = 0
    adp_lasso_bic_mse_spr = 0
    adp_lasso_loocv_mse_spr = 0
    
    ridge_aic_mse_spr = 0
    ridge_bic_mse_spr = 0
    ridge_loocv_mse_spr = 0
    adp_ridge_aic_mse_spr = 0
    adp_ridge_bic_mse_spr = 0
    adp_ridge_loocv_mse_spr = 0
    
    # Dense Signal 
    lasso_aic_mse_dns = 0
    lasso_bic_mse_dns = 0
    lasso_loocv_mse_dns = 0
    adp_lasso_aic_mse_dns = 0
    adp_lasso_bic_mse_dns = 0
    adp_lasso_loocv_mse_dns = 0
    
    ridge_aic_mse_dns = 0
    ridge_bic_mse_dns = 0
    ridge_loocv_mse_dns = 0
    adp_ridge_aic_mse_dns = 0
    adp_ridge_bic_mse_dns = 0
    adp_ridge_loocv_mse_dns = 0
    
    # For LOO-CV
    loo = LeaveOneOut()
    
    prog = tqdm(total = 1000, position=0, leave=2)
    
    for _ in range(1000):
        # Create Dataset
        epsilon_spr = np.random.normal(loc=0, scale=sigma2_spr)
        epsilon_dns = np.random.normal(loc=0, scale=sigma2_dns)

        X = np.random.multivariate_normal(mean=np.zeros(p), cov=Sigma_p, size=n)

        Y_spr =  X @ beta_spr + epsilon_spr
        Y_dns =  X @ beta_dns + epsilon_dns

        # For adaptive regressions
        lm_spr = LinearRegression().fit(X, Y_spr)
        lm_dns = LinearRegression().fit(X, Y_dns)
        
        # Adpative Lasso
        # Here we are compute X^T (D^-1)^T
        adX_sprL = X @ np.diagflat(np.abs(lm_spr.coef_))
        adX_dnsL = X @ np.diagflat(np.abs(lm_dns.coef_))
        
        # Adpative Ridge
        adX_spr_Rd = X @ np.diagflat((lm_spr.coef_**2)**(1/2))
        adX_dns_Rd = X @ np.diagflat((lm_dns.coef_**2)**(1/2))
    
        # Calculate Lassos
        
        # AIC
        
        # Sparse
        lasso_aic_spr = LassoLarsIC(criterion='aic')
        lasso_aic_spr.fit(X, Y_spr)
        lasso_aic_mse_spr += mean_squared_error(Y_spr, lasso_aic_spr.predict(X))
        
        # Dense
        lasso_aic_dns = LassoLarsIC(criterion='aic')
        lasso_aic_dns.fit(X, Y_dns)
        lasso_aic_mse_dns += mean_squared_error(Y_dns, lasso_aic_dns.predict(X))
        
        #BIC
        
        # Sparse
        lasso_bic_spr = LassoLarsIC(criterion='bic')
        lasso_bic_spr.fit(X, Y_spr)
        lasso_bic_mse_spr += mean_squared_error(Y_spr, lasso_bic_spr.predict(X))
        
        # Dense
        lasso_bic_dns = LassoLarsIC(criterion='bic')
        lasso_bic_dns.fit(X, Y_dns)
        lasso_bic_mse_dns += mean_squared_error(Y_dns, lasso_bic_dns.predict(X))
        
        #LOO-CV
        
        # Sparse
        lasso_loo_spr = LassoLarsCV(cv=loo)
        lasso_loo_spr.fit(X, Y_spr)
        lasso_loocv_mse_spr += mean_squared_error(Y_spr, lasso_loo_spr.predict(X))
        
        # Dense
        lasso_loo_dns = LassoLarsCV(cv=loo)
        lasso_loo_dns.fit(X, Y_dns)
        lasso_loocv_mse_spr += mean_squared_error(Y_dns, lasso_loo_dns.predict(X))

        # Adaptive Lasso
        
        # AIC
        
        # Sparse
        adp_lasso_aic_spr = LassoLarsIC(criterion='aic')
        adp_lasso_aic_spr.fit(adX_sprL, Y_spr)
        adp_lasso_aic_mse_spr += mean_squared_error(Y_spr, adp_lasso_aic_spr.predict(adX_sprL))
        
        # Dense
        adp_lasso_aic_dns = LassoLarsIC(criterion='aic')
        adp_lasso_aic_dns.fit(adX_dnsL, Y_dns)
        adp_lasso_aic_mse_dns += mean_squared_error(Y_dns, adp_lasso_aic_dns.predict(adX_dnsL))
        
        # BIC
        
        # Sparse
        adp_lasso_bic_spr = LassoLarsIC(criterion='bic')
        adp_lasso_bic_spr.fit(adX_sprL, Y_spr)
        adp_lasso_bic_mse_spr += mean_squared_error(Y_spr, adp_lasso_bic_spr.predict(adX_sprL))
        
        # Dense
        adp_lasso_bic_dns = LassoLarsIC(criterion='bic')
        adp_lasso_bic_dns.fit(adX_dnsL, Y_dns)
        adp_lasso_bic_mse_dns += mean_squared_error(Y_dns, adp_lasso_bic_dns.predict(adX_dnsL))
        
        # LOO-CV
        
        # Sparse
        #adp_lasso_loo_spr = LassoLarsCV(cv=loo)
        #adp_lasso_loo_spr.fit(adX_sprL, Y_spr)
        #adp_lasso_loocv_mse_spr += mean_squared_error(Y_spr, adp_lasso_loo_spr.predict(adX_sprL))
        
        # Dense
        #adp_lasso_loo_dns = LassoLarsCV(cv=loo)
        #adp_lasso_loo_dns.fit(adX_dnsL, Y_dns)
        #adp_lasso_loocv_mse_dns += mean_squared_error(Y_dns, adp_lasso_loo_dns.predict(adX_dnsL))

        # Calculate Ridge
        # AIC

        #BIC

        #LOO-CV

        #Adaptive Ridge
        # AIC

        # BIC

        # LOO-CV
        
        prog.update()


  0%|          | 0/1000 [00:00<?, ?it/s]

10 0


 88%|████████▊ | 884/1000 [03:06<00:24,  4.75it/s]

In [37]:
n= 100
p = 10
rho = .25
beta_spr = np.fromfunction(lambda j,_: (2/np.sqrt(n))*(j+1 <= np.sqrt(p)), (p,1))

Sigma_p = np.fromfunction(lambda i,j: rho**(np.abs(i-j)), (p,p))
sigma2_spr = .25*(beta_spr.T @ Sigma_p @ beta_spr)

epsilon_spr = np.random.normal(loc=0, scale=sigma2_spr)

X = np.random.multivariate_normal(mean = np.zeros(p), cov = Sigma_p, size = n)

Y_spr =  X @ beta_sp + epsilon_spr


lasso_aic_spr = LassoLarsIC(criterion='aic')
x = lasso_aic_spr.fit(X, Y_spr.ravel())

In [39]:
 #For addpative regressions
lm_spr = LinearRegression().fit(X, Y_spr)
lm_dns = LinearRegression().fit(X, Y_dns)
        
        # Adpative Lasso
        # Here we are compute X^T (D^-1)^T
adX_sprL = X @ np.diagflat(np.abs(lm_spr.coef_))
adX_dnsL = X @ np.diagflat(np.abs(lm_dns.coef_))
        
        # Adpative Ridge
adX_spr_Rd = X @ np.diagflat((lm_spr.coef_**2)**(1/2))
adX_dns_Rd = X @ np.diagflat((lm_dns.coef_**2)**(1/2)) 

In [40]:
lm_spr.coef_

array([[ 2.00000000e-01,  2.00000000e-01,  2.00000000e-01,
         2.41351772e-17,  1.81752026e-17, -1.92174004e-17,
        -9.49103888e-17, -1.79067230e-16, -1.83295070e-16,
        -8.29886526e-17]])

In [41]:
np.diagflat(np.abs(lm_spr.coef_))

array([[2.00000000e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 2.00000000e-01, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 2.00000000e-01, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.41351772e-17,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.81752026e-17, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
   

In [43]:
x = []

In [54]:
x.append(5)
x

[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]